### Imports

In [1]:
from osgeo import ogr, osr
import os
import base64 # for encoding the image (SVG)
import re

import glob


### Files

In [2]:
s57_folder = "GIS_files/11CGD_ENCs/ENC_ROOT"
s57_all_encs = 'GIS_files/All_ENCs'
geopackage_export_folder = 'GIS_export/ENC2/'
enc_dvd = "G:/Python Projects/ENC Files/DVD_1"


Converting S57 to GeoPackage

In [5]:
os.environ['OGR_S57_OPTIONS'] = '''RETURN_PRIMITIVES=OFF, 
                                   SPLIT_MULTIPOINT=ON,
                                   ADD_SOUNDG_DEPTH=ON,
                                   RETURN_LINKAGES=ON, 
                                   RECODE_BY_DSSI=ON'''

def is_empty_or_null(value):
    if value is None:
        return True
    if isinstance(value, str):
        return value.strip() == "" or value.strip().lower() in ["null", "none"]
    return False



def convert_s57_to_geopackage(s57_file_path, geopackage_export_folder):
    """
    Convert an S-57 ENC file to GeoPackage format.

    Parameters:
    s57_file_path (str): Path to the input S-57 file.
    geopackage_path (str): Path to the output GeoPackage file.

    Returns:
    None
    """
    # Extract the base name of the S57 file (without extension)
    s57_base_name = os.path.splitext(os.path.basename(s57_file_path))[0]
    
    # Create the full path for the output GeoPackage directly in the export folder
    geopackage_export_path = os.path.join(geopackage_export_folder, f"{s57_base_name}.gpkg")
    
    # Ensure the export folder exists
    os.makedirs(geopackage_export_folder, exist_ok=True)

    print(f"Attempting to create GeoPackage at: {geopackage_export_path}")
    # Register drivers
    ogr.RegisterAll()
    s57_driver = ogr.GetDriverByName('S57')
    gpkg_driver = ogr.GetDriverByName('GPKG')

    # Open the S-57 file
    s57 = s57_driver.Open(s57_file_path)
    if s57 is None:
        raise RuntimeError(f"Could not open S-57 file: {s57_file_path}")
    else:
        print(f"S-57 file opened: {s57_file_path}")

    # Create the GeoPackage file
    gpkg = gpkg_driver.CreateDataSource(geopackage_export_path)
    if gpkg is None:
        raise RuntimeError(f"Could not create GeoPackage: {geopackage_export_path}")
    else:
        print(f"GeoPackage created: {geopackage_export_path}")

    # Get the number of layers in the S-57 file
    num_layers = s57.GetLayerCount()
    print(f"Number of layers in S-57 file: {num_layers}")

    # # Check if the S-57 projection is EPSG:4326 (WGS84)
    
    # print(f"S-57 spatial reference system: {srs_s57}")
    # if srs_s57 is not None:
    #     if srs.IsSame(osr.SpatialReference().ImportFromEPSG(4326)):
    #         print("S-57 projection is EPSG:4326 (WGS84)")
    #     else:
    #         print("S-57 projection is not EPSG:4326")
    # else:
    #     print("Spatial reference system is not defined")
    

    # Loop through the layers in the S-57 file
    for i in range(num_layers):
        layer = s57.GetLayer(i)
        layer_name = layer.GetName()
        layer_defn = layer.GetLayerDefn()
        print(f"\nProcessing S-57 layer: {layer_name} in {s57_base_name} ENC")

        # Get the spatial reference system of the layer
        srs = layer.GetSpatialRef()
        if srs:
            # Get the EPSG code
            epsg_code = srs.GetAuthorityCode(None)
            # Get the projection name
            datum_name = srs.GetAttrValue('DATUM')
            proj_name = srs.GetAttrValue('PROJECTION')
            units = srs.GetAttrValue('UNIT')
            
            print(f"Layer {i+1}: {layer_name}")
            print(f"EPSG Code: {epsg_code}")
            print(f"Datum: {datum_name if datum_name else 'Not specified'}")
            print(f"Projection: {proj_name if proj_name else 'Not specified'}")
            print(f"Units: {units if units else 'Not specified'}")
        else:
            print(f"\nLayer {i+1}: {layer_name}")
            print("No spatial reference system defined")

        # Create the corresponding layer in the GeoPackage file
        gpkg_layer = gpkg.CreateLayer(layer_name, srs, layer.GetGeomType())
        if gpkg_layer is None:
            raise RuntimeError(f"Could not create layer: {layer_name}")
        else:
            print(f"GeoPackage layer created: {layer_name}")

        # Add fields to the GeoPackage layer
        for i in range(layer_defn.GetFieldCount()):
            s57_field_defn = layer_defn.GetFieldDefn(i)
            field_name = s57_field_defn.GetName()
            field_type = s57_field_defn.GetType()
            
            field_defn = ogr.FieldDefn(field_name, field_type)
            gpkg_layer.CreateField(field_defn)

        # Get the number of features in the layer
        num_features = layer.GetFeatureCount()
        print(f"Number of features in {layer_name} layer: {num_features}")
        
        """This approach uses the layer's GetNextFeature() method to iterate through 
        features, which is more reliable than using indices. It will properly iterate 
        through all available features and their fields, avoiding the "Skipping null 
        feature" issue for valid features."""

        # Loop through the features in the layer
        layer.ResetReading()  # Reset the reading position to the start of the layer
        feature = layer.GetNextFeature()


        while feature:
            feature_id = feature.GetFID()
            print(f"Processing feature ID: {feature_id}")

            # Get the feature geometry
            geometry = feature.GetGeometryRef()

            # Create the feature in the GeoPackage layer
            gpkg_feature = ogr.Feature(gpkg_layer.GetLayerDefn())

            if layer_name != "DSID":
                if geometry is None:
                    print(f"Feature {feature_id} has no geometry, skipping")
                    feature = layer.GetNextFeature()
                    continue
                print(f"Processing {geometry.GetGeometryName()} geometry...")
                gpkg_feature.SetGeometry(geometry)

            # Loop through the fields in the feature
            for i in range(feature.GetFieldCount()):
                field_def = feature.GetFieldDefnRef(i)
                field_name = field_def.GetName()
                field_type = field_defn.GetType()
                field_value = feature.GetFieldAsString(field_name)
                
                # Check if the field is NULL in S-57
                if is_empty_or_null(field_value):
                    gpkg_feature.SetFieldNull(i)
                else: 
                    gpkg_feature.SetField(field_name, field_value)

            # Add the feature to the GeoPackage layer
            if gpkg_layer.CreateFeature(gpkg_feature) != 0:
                raise RuntimeError(f"Failed to create feature {feature_id} in layer {layer_name}")
            
            print(f"Feature {feature_id} created in layer: {layer_name}")

            if layer.GetName() == "DSID":
                edition = feature.GetFieldAsString("DSID_EDTN")
                update = feature.GetFieldAsString("DSID_UPDN")
                print(f"ENC {s57_base_name} EDITION: {edition}")
                print(f"ENC {s57_base_name} UPDATE: {update}")

            # Get the next feature
            feature = layer.GetNextFeature()
        
        # Compare None values
        field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]
        for s57_feature, gpkg_feature in zip(layer, gpkg_layer):
            for i, field_name in enumerate(field_names):
                s57_field_type = s57_feature.GetFieldDefnRef(field_name).GetType()
                
                # Handle list types correctly
                if s57_field_type in [ogr.OFTStringList, ogr.OFTIntegerList, ogr.OFTRealList]:
                    # Retrieve the correct list based on the field type
                    if s57_field_type == ogr.OFTStringList:
                        get_value = s57_feature.GetFieldAsStringList(i)
                    elif s57_field_type == ogr.OFTIntegerList:
                        get_value = s57_feature.GetFieldAsIntegerList(i)
                    elif s57_field_type == ogr.OFTRealList:
                        get_value = s57_feature.GetFieldAsDoubleList(i)

                    if get_value:
                        # Convert all elements to strings (if needed)
                        string_list = ",".join([str(item) for item in get_value])
                        gpkg_feature.SetField(i, string_list)
                        

                s57_value = s57_feature.GetField(field_name)
                gpkg_value = gpkg_feature.GetField(field_name)
                if s57_value is None and gpkg_value is not None:
                    print(f"Mismatch in {field_name} ({s57_field_type}): S-57 None, GeoPackage {gpkg_value}")
                elif s57_value is not None and gpkg_value is None:
                    print(f"Mismatch in {field_name} ({s57_field_type}): S-57 {s57_value}, GeoPackage None")
                    
                        
        # Reset the layer reading position after processing
        layer.ResetReading()
    # Close the dataset
    gpkg = None
    s57 = None

# Browse the ENC_ROOT folder and process all S-57 files
def process_enc_folder(enc_folder_path, geopackage_export_folder):
    s57_files = []
    for root, dirs, files in os.walk(enc_folder_path):
        for path in glob.glob(os.path.join(root, '*.000')):
            s57_files.extend([os.path.normpath(path) for path in glob.glob(os.path.join(root, '*.[0-9][0-9][0-9]'))])
    
    print(f"Found {len(s57_files)} S-57 files in folder: {enc_folder_path}")
    if s57_files:
        print(s57_files)
        for s57_file in s57_files:
            print(f"\nProcessing S-57 file: {s57_file}")
            convert_s57_to_geopackage(s57_file, geopackage_export_folder)  

# Example usage
s57_folder = "GIS_files/07CGD_ENCs/ENC_ROOT"
geopackage_export_folder = 'GIS_export/ENC2/'
process_enc_folder(s57_folder, geopackage_export_folder)

Found 842 S-57 files in folder: GIS_files/07CGD_ENCs/ENC_ROOT
['GIS_files\\07CGD_ENCs\\ENC_ROOT\\US1GC09M\\US1GC09M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US1GC09M\\US1GC09M.001', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US1GC09M\\US1GC09M.002', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US1GC09M\\US1GC09M.003', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US2EC01M\\US2EC01M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US2EC01M\\US2EC01M.001', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US2EC01M\\US2EC01M.002', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US2EC02M\\US2EC02M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US2GC09M\\US2GC09M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3FL28M\\US3FL28M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3FL28M\\US3FL28M.001', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3FL30M\\US3FL30M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3FL90M\\US3FL90M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3GA10M\\US3GA10M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3GC06M\\US3GC06M.000', 'GIS_files\\07CGD_ENCs\\ENC_ROOT\\US3GC07M\\US3GC07M

RuntimeError: Could not open S-57 file: GIS_files\07CGD_ENCs\ENC_ROOT\US1GC09M\US1GC09M.001

Check GeoPackage Fields

In [ ]:
# Check GeoPackage data

def  get_field_data(geopackage_export_folder, enc_name, layer_name, field_name):

    geopackage_path  = os.path.join(geopackage_export_folder, f"{enc_name}.gpkg")

    # Check if the GeoPackage file exists
    if not os.path.exists(geopackage_path):
        raise FileNotFoundError(f"GeoPackage with {enc_name} name, not found in {geopackage_export_folder}")

    # Register the GeoPackage driver
    ogr.RegisterAll()
    gpkg_driver = ogr.GetDriverByName('GPKG')

    # Open the GeoPackage file
    gpkg = gpkg_driver.Open(geopackage_path)
    if gpkg is None:
        raise RuntimeError(f"Could not open GeoPackage: {geopackage_path}")
    else:
        print(f"GeoPackage opened: {geopackage_path}")

    # Get the layer by name
    layer = gpkg.GetLayerByName(layer_name)
    if layer is None:
        raise RuntimeError(f"Layer not found: {layer_name}")
    else:
        print(f"Layer found: {layer_name} with {layer.GetFeatureCount()} features")

    # Loop through the features in the layer
    for feature in layer:
        # Get the field value
        field_value = feature.GetField(field_name)
        print(f"Field: {field_name}, Value: {field_value}")

    # Close the dataset
    gpkg = None

# Example usage
#get_field_data(geopackage_export_folder, "US1WC01M",'BCNLAT','FFPT_RIND')